In [ ]:
import requests
import numpy as np
import pandas as pd


# Create function to get raw json return from FRED database
def get_series_json(series_id, start, end, api_key='fbf2a3cac76ec733ee2b8c01ab036950'
                , file_type='json'):
    url_base = 'https://api.stlouisfed.org/fred/series/observations'
    url = f'{url_base}?series_id={series_id}'
    if start is not None:
        start = pd.to_datetime(start, errors='raise')
        url += '&observation_start=' + start.strftime('%Y-%m-%d')
    if end is not None:
        end = pd.to_datetime(end, errors='raise')
        url += '&observation_end=' + end.strftime('%Y-%m-%d')
    url += f'&api_key={api_key}&file_type={file_type}'
    
    try:
        resp = requests.get(url)
        resp.raise_for_status()  # Raise exception if invalid response
        return resp
    except Exception as e:
        errmsg = resp.json()['error_message'].replace('series', f'series {series_id}')
        print(f'Error: {resp.status_code}\n{errmsg}')
        return None
    

    
# Create function to transform valid json response from FRED into a dataframe
def transform_series_json(resp, series_id):
    resp = resp.json()
    obs = pd.DataFrame(resp.pop('observations'))[['date', 'value']]
    obs['date'] = pd.to_datetime(obs['date'])
    obs.set_index('date', inplace=True)
    
    # meta = pd.DataFrame({series_id: resp})
    # meta.loc['NaNs'] = obs.value_counts()['.']  # Count missing values and add to meta df
    meta = pd.DataFrame({
        series_id: {
            'observation_start': resp['observation_start'],
            'observation_end': resp['observation_end'],
            'count': resp['count'],
            'actual_start': obs.index.min().date(),
            'actual_end': obs.index.max().date(),
            'NaN count': obs[obs.value == '.'].count().value,
        }
    })
    
    obs.loc[obs.value == '.'] = np.nan
    obs.columns = [series_id]
    obs[series_id] = obs[series_id].astype(float, errors='raise')
    
    return obs, meta
    
# Create function to fill missing values in FRED series datafame
def fill_series_na(df):
    df.fillna(method='ffill', inplace=True)  # Fill missing values with last observation
    df.fillna(method='bfill', inplace=True)  # Then, fill with next observation
    
    return df

# Create a function to get a time series from FRED and return a clean dataframe
def get_series(series_id, start, end, api_key=None, file_type='json', fill_na=None, return_meta=None):
    fill_na = True if fill_na is None else fill_na  # Default
    return_meta = False if return_meta is None else return_meta  # Default
    api_key = 'fbf2a3cac76ec733ee2b8c01ab036950' if api_key is None else api_key # Default
    
    try:
        resp = get_series_json(series_id=series_id, start=start, end=end, api_key=api_key, file_type=file_type)
        df, meta = transform_series_json(resp, series_id=series_id)
        # df = pd.DataFrame.from_dict(df, orient='index', columns=[series_id])
        df = fill_series_na(df) if fill_na else df
    except Exception as e:
        print(f'Error: {e}')
        return None
    
    return df, meta
               

series = 'DEXUSUK'
# series = 'DEXUKUS'  # Incorrect series ID
# series = 'DEXUSEU'  # EUR/USD, starts in 1999
# series = 'DEXSFUS'  # USD/ZAR, no '.' (NaNs)
start = '1983-01-01'
end = '2022-12-31'

df, meta = get_series(series_id=series, start=start, end=end, fill_na=False)